In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
import config
import numpy as np

from ticker_analysis import finance
from ticker_analysis.finance import FinanceApp
from ticker_analysis.stock_data_handler import StockDataHandler

from stochastic_recurrent_neural_network import StochasticRecurrentNeuralNetwork
from neural_network import NeuralNetwork
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [18]:
stock = StockDataHandler(ticker='AAPL', 
                         period='daily_adjusted', 
                         api_key=config.ALPHAVANTAGE_API_KEY, 
                         start_date='2004-01-01', 
                         end_date='2022-12-31')
df = stock.get_data()

In [20]:
app = FinanceApp()
app.load_stock_data(df)
app.show()

In [23]:
df.columns

Index(['1. open', '2. high', '3. low', '4. close', '5. adjusted close',
       '6. volume', '7. dividend amount', '8. split coefficient', 'return',
       'volatility'],
      dtype='object')

In [24]:

# Prepare the data for Neural Network
# Here we will try to predict the Close price
X = df.drop('4. close', axis=1).values
y = df['4. close'].values

# Normalize the data
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
y = y.reshape(-1, 1)
y = scaler.fit_transform(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the input size, hidden size and output size
input_size = X_train.shape[1]  # Number of features
hidden_size = 10
output_size = 1  # We are predicting the Close price

nn = NeuralNetwork(input_size=input_size, hidden_size=hidden_size, output_size=output_size, dropout_rate=0.2)

epochs = 100
learning_rate = 0.01

best_val_loss = np.inf
patience = 10
epochs_without_improvement = 0

for epoch in range(epochs):
    # During training, set is_training=True
    A1, A2 = nn.forward_pass(X_train, is_training=True)
    dW1, db1, dW2, db2 = nn.backward_pass(X_train, y_train)
    nn.update_weights(dW1, db1, dW2, db2, learning_rate)

    # Compute validation loss at each epoch
    A1_val, A2_val = nn.forward_pass(X_val, is_training=False)
    val_loss = nn.compute_loss(y_val, A2_val)

    # If validation loss is lower than the best loss seen so far,
    # then this is the best model we've seen so far
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        epochs_without_improvement = 0
    else:
        # If validation loss didn't improve, increment counter
        epochs_without_improvement += 1

    if epochs_without_improvement >= patience:
        print("Early stopping")
        break


Early stopping
